Updated training notebook. Now much faster and easier than before. Maintained by DYVAUX.

Notebook is not ready for use.

Be sure to click the <img src='https://i.imgur.com/IfKbEiK.png'> icon to view files.

In [ ]:
# Connect Google Drive
from google.colab import drive
print('\033[97;100m' + 'Mounting Google Drive.' + '\033[0m')
drive.mount("/content/drive")

In [ ]:
# Install NNSVS
%rm -rf sample_data
%cd -q /content
!git clone https://github.com/DYVAUX/enunu_training_kit.git ETK &> /dev/null
!bash /content/ETK/train/install.sh
!ln -sf /content/ETK/train/conf/train/acoustic/model/acoustic_conv1dresnet.yaml /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml &> /dev/null
!ln -sf /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml ACOUSTIC_SETTINGS.yaml &> /dev/null
%reload_ext tensorboard

In [ ]:
# Decompress your dataset

empty_dataset_folder = False #@param {type:"boolean"}
compressed_dataset_path = "/content/dataset.*" #@param {type:"string"}

if empty_dataset_folder == True:
  !rm -rf /content/ETK/singing_database

!7za  -bso0 -y x "$compressed_dataset_path" -o/content/ETK/singing_database/
print('\nDone!')

#@markdown If you choose to not use this cell, upload your files to `ETK/singing_database/`

#Configure NNSVS
---

In [ ]:
#@title Set Language
lang = "Japanese" #@param ["Japanese", "English", "Romance", "Chinese", "Polish", "Custom"]

#@markdown if custom language, set these.
custom_hed = "custom.hed" #@param {type:"string"}
custom_table = "custom.table" #@param {type:"string"}
vowels = "a, e ,i, o, u, N, A, E, I, O, U" #@param {type:"string"}

#acoustic model type:
model = "Conv1dResnet" #@param ["Conv1dResnet", "ResF0Conv1dResnet", "ResSkipF0FFConvLSTM", "VariancePredictor", "ResF0VariancePredictor", "ResF0NonAttentiveTacotron"]
use_mdn = False #@param {type:"boolean"}
#@markdown MDN is available for all acoustic model types except Tacotron. MDN is not recommended below 1hr of human audio.

vibrato =  "none" #@param ["none", "diff", "sine"]
# change out_dim to 206 when using vibrato
# might require ResF0
# add stream sized to acoustic model config (note: [180, 3, 1, 15], diff: [180, 3, 1, 15, 3], sine: [180, 3, 1, 15, 6, 1])
# change acosutic features file
# this may be broken and I don't know how to fix it right now.
# may remove "diff" as an option.

sample_rate = "44100" #@param ["44100", "48000"]

# so the notebook is more likely to work on Jupyter.
import importlib
nnsvs_path = importlib.util.find_spec("nnsvs")
nnsvs_path = nnsvs_path.submodule_search_locations[0]
print(nnsvs_path)

def get_vowels():
  import subprocess
  vowels = subprocess.getoutput('sed -n -r \'s/^.*("C-Vowel"|"C-Vowels"|"C-Phone_Boin")\s+//p\' /content/ETK/train/hed/'+hed_file)
  junk = ['{', '}', '-', '+', '*']
  table = str.maketrans('', '', ''.join(junk))
  vowels = vowels.translate(table)
  vowels = vowels.replace(",","', '")
  print(vowels)
  !sed -i -r "s|(VOWELS = )(.+)|\1'{vowels}'|g" /content/ETK/train/stage0/compare_mono_align_and_mono_score.py
    
def get_in_dim():
    file  = open("/content/ETK/train/hed/"+hed_file, 'r').read()
    in_dim = file.count("QS")
    acoustic_in_dim = in_dim + int(4)
    print("in_dim:", + in_dim)
    print("acoustic_in_dim:", + acoustic_in_dim)
    !sed -i -r 's|(in_dim:)(\s+)(.+)|\1\2{in_dim}|g' /content/ETK/train/conf/train/timelag/model/timelag_mdnv2.yaml
    !sed -i -r 's|(in_dim:)(\s+)(.+)|\1\2{in_dim}|g' /content/ETK/train/conf/train/duration/model/duration_mdnv2.yaml
    !sed -i -r 's|(in_dim:)(\s+)(.+)|\1\2{acoustic_in_dim}|g' /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml

def get_idx():
    from nnmnkwii.io import hts
    binary_dict, continuous_dict = hts.load_question_set("/content/ETK/train/hed/" + hed_file)
    for n in range(len(binary_dict)):
        if binary_dict[n][0] in ("C-Silence", "C-Silences", "C-Phone_Muon"):
            in_rest_idx = n
            print("in_rest_idx:", in_rest_idx)
            !sed -i -r 's|(in_rest_idx:)(\s+)(.+)|\1\2{in_rest_idx}|g' /content/ETK/train/conf/train/acoustic/data/myconfig.yaml
    for n in range(len(continuous_dict)):
        if continuous_dict[n][0] in ("e1", "e1_absolute_pitch"): # the absolute pitch of the current note 
            in_lf0_idx = n+  len(binary_dict)
            print("in_lf0_idx:", in_lf0_idx)
            !sed -i -r 's|(in_lf0_idx:)(\s+)(.+)|\1\2{in_lf0_idx}|g' /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml
            !sed -i -r 's|(in_lf0_idx:)(\s+)(.+)|\1\2{in_lf0_idx}|g' /content/ETK/train/conf/train/acoustic/data/myconfig.yaml

!ln -sf /content/ETK/train/conf/train/acoustic/model/acoustic_{model.lower()}.yaml /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml &> /dev/null

#These will pull the "main" branch for each language to reduce maintanance.
if lang == "Japanese":
    print("Switching to DYVAUX Japanese")
    hed_file = "DVX_jp.hed"
    dic_file = "DVX_jp.table"
elif lang == "English":
    print("Switching to DYVAUX English")
    !git clone https://github.com/DYVAUX/nnsvs-english-support lang-ext &> /dev/null
    hed_file = "DVX_en.hed"
    dic_file = "DVX_en.table"
elif lang == "Romance":
    print("Switching to DYVAUX Romance Family")
    !git clone https://github.com/DYVAUX/nnsvs-romance-language-support lang-ext &> /dev/null
    hed_file = "DVX_lat.hed"
    dic_file = "DVX_lat.table"
elif lang == "CJK":
    print("Switching to DYVAUX CJK")
    hed_file = "DVX_cjk.hed"
    dic_file = "DVX_cjk.table"
    print("SELECTED LANGUAGE IS UNAVAILABLE")
elif lang == "Chinese":
    print("Switching to Archivoice Chinese")
    !git clone https://github.com/Archivoice/nnsvs-chinese-support lang-ext &> /dev/null
    hed_file = "chinese.hed"
    dic_file = "chinese.table"
elif lang == "Polish":
    print("Switching to SzTP Polish")
    !git clone https://github.com/SzopaTatyJarka/nnsvs-polish-support lang-ext &> /dev/null
    hed_file = "sztj_polish.hed"
    dic_file = "sztj_polish_phoneme.table"
elif lang == "Custom":
    print("Switching to Custom Language")
    hed_file = custom_hed
    dic_file = custom_table
!cp -r lang-ext/hed /content/ETK/train &> /dev/null
!cp -r lang-ext/dic /content/ETK/train &> /dev/null
!rm -rf lang-ext
    
get_in_dim()
if lang == 'Japanese':
  vowels = "'a', 'i', 'u', 'e', 'o', 'A', 'I', 'U', 'E', 'O', 'N'"
  print(vowels)
  !sed -i -r "s|(VOWELS = )(.+)|\1{vowels}|g" /content/ETK/train/stage0/compare_mono_align_and_mono_score.py
elif lang == 'Custom':
  print(vowels)
  !sed -i -r "s|(VOWELS = )(.+)|\1{vowels}|g" /content/ETK/train/stage0/compare_mono_align_and_mono_score.py
else:
  get_vowels()

if model.startswith('Res'):
    print("Model starts with Res")
    get_idx()
    !sed -i -r 's|(relative_f0:)(\s+)(.+)|\1\2false|g' /content/ETK/train/enuconfig.yaml
    !sed -i -r 's|(relative_f0:)(\s+)(.+)|\1\2false|g' {nnsvs_path}/bin/conf/prepare_features/acoustic/static_*.yaml
    !sed -i -r 's|(out_lf0_idx:)(\s+)(.+)|\1\2180|g' /content/ETK/train/conf/train/acoustic/data/myconfig.yaml
    # change to nnsvs-train-resf0
else:
    print("Model doesn't start with Res")
    # Set relative_f0 true static_deltadelta_* and enuconfig
    # change to nnsvs-train
    # set idx to null
    !sed -i -r 's|(in_rest_idx:)(\s+)(.+)|\1\2null|g' /content/ETK/train/conf/train/acoustic/data/myconfig.yaml
    !sed -i -r 's|(in_lf0_idx:)(\s+)(.+)|\1\2null|g' /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml
    !sed -i -r 's|(in_lf0_idx:)(\s+)(.+)|\1\2null|g' /content/ETK/train/conf/train/acoustic/data/myconfig.yaml
    !sed -i -r 's|(out_lf0_idx:)(\s+)(.+)|\1\2null|g' /content/ETK/train/conf/train/acoustic/data/myconfig.yaml

if use_mdn:
  !sed -i -r 's|(use_mdn:)(\s+)(.+)|\1\2true|g' /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml
else:
  !sed -i -r 's|(use_mdn:)(\s+)(.+)|\1\2false|g' /content/ETK/train/conf/train/acoustic/model/acoustic_current.yaml

!sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/{hed_file}|g' /content/ETK/train/enuconfig.yaml
!sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/{dic_file}|g' /content/ETK/train/config.yaml
!sed -i -r 's|(question_path:)(\s+)(.+)|\1\2hed/{hed_file}|g' /content/ETK/train/config.yaml
!sed -i -r 's|(table_path:)(\s+)(.+)|\1\2dic/{dic_file}|g' /content/ETK/train/enuconfig.yaml
#sed vowel list into compare_mono_align_and_mono_score

print("End")

In [ ]:
#@title Set parameters

pretained_exp = ""

#@markdown The notebook uses sane defaults for all model types. Additional settings removed to make training more simple.
acoustic_epochs = 120 #@param {type:"integer"}
acoustic_loss = "mse" #@param ["mse", "mae"]
acoustic_init = "none" #@param ["none", "kaiming_normal", "xavier_normal"]
#@markdown Due to to the difference in training parameters, you can change advanced settings in the `acoustic_current.yaml file`

#The notebook uses MDNv2 by default with optimal settings. Choose desired number of epochs.
duration_epochs = 200 #@param {type:"integer"}
duration_init = "none" #@param ["none", "kaiming_normal", "xavier_normal"]
timelag_epochs = 200 #@param {type:"integer"}
timelag_init = "none" #@param ["none", "kaiming_normal", "xavier_normal"]

d4c_threshold = 0.85 #@param {type:"slider", min:0, max:1, step:0.01}
pitch_reg_weight = 0.6 #@param {type:"number"}
# will need to update in the correct acoustic features file. Or change all of them.

In [ ]:
#@markdown Tensorboard.
#@markdown View training info. non-live.

import datetime, os
%tensorboard --load_fast=true --reload_interval=1 --reload_multifile=true --logdir=/content/enunu_training_kit-main/train/tensorboard/

In [ ]:
#steps to train the main models 0 - 5

starting_stage = 4 #@param {type:"slider", min:0, max:7, step:1.0}
stopping_stage = 4 #@param {type:"slider", min:0, max:7, step:1.0}
singer_name = "Unnamed_Singer" #@param {type:"string"}
#@markdown No spaces in `singer_name:`. Use underscores `_`
!sed -i -r 's|(spk:)(\s+)(.+)|\1\2{singer_name}|g' /content/ETK/train/config.yaml
!sed -i -r 's|(model_dir:)(\s+)(.+)|\1\2exp/{singer_name}\_dyvaux_notebook|g' /content/ETK/train/enuconfig.yaml
!sed -i -r 's|(stats_dir:)(\s+)(.+)|\1\2dump/{singer_name}/norm|g' /content/ETK/train/enuconfig.yaml

%cd "/content/ETK/train"
!bash run.sh --stage $starting_stage --stop_stage $stopping_stage

In [ ]:
#steps to train postfilter

print("training postfilter for " + singer_name)
starting_stage = 7 #@param {type:"slider", min:6, max:7, step:1.0}
stopping_stage = 7 #@param {type:"slider", min:6, max:7, step:1.0}

%cd "/content/ETK/train"
!bash run.sh --stage $starting_stage --stop_stage $stopping_stage

In [ ]:
#steps to train vocoder
print("this doesn't do anything, sorry!")

#print("training vocoder for " + singer_name)
#starting_stage = 8 #@param {type:"slider", min:8, max:10, step:1.0}
#stopping_stage = 10 #@param {type:"slider", min:8, max:10, step:1.0}

#%cd "/content/ETK/train"
#!bash run.sh --stage $starting_stage --stop_stage $stopping_stage

In [ ]:
#release step and 7z
print("Packaging " + singer_name)
%cd "/content/ETK/train"
!bash run.sh --stage 11 --stop_stage 11

store_on_google_drive = False #@param {type:"boolean"}

delete_checkpoint = True #@param {type:"boolean"}
#@markdown `delete_checkpoint` deletes extra checkpoints to reduce singer size. Does not affect quality.

import subprocess
time = subprocess.getoutput('date +%y%m%d-%H%M-UTC')

%cd "/content/ETK/train"
!bash run.sh --stage 11 --stop_stage 11

if delete_checkpoint:
    %rm -f /content/ETK/train/release/{singer_name}_---/exp/{singer_name}_dyvaux_notebook/*/checkpoint*
    %rm -f /content/ETK/train/release/{singer_name}_---/exp/{singer_name}_dyvaux_notebook/*/epoch*

%cd -q "/content/ETK/train/release/"
print('\033[97;100m' + 'Compressing Model' + '\033[0m')
!7za -bso0 a "/content/{singer_name}_NNSVS_model_{time}.7z" "{singer_name}_---"
if store_on_google_drive == True:
    %mkdir -p /content/drive/MyDrive/NNSVS_Release_Models
    print('\033[97;100m' + 'Copying Singer to your Google Drive' + '\033[0m')
    !cp /content/{singer_name}_NNSVS_model_{time}.7z > /content/drive/MyDrive/NNSVS_Release_Models/{singer_name}_NNSVS_model_{time}.7z
    print('Done!')
else:
    print('Done!')